In [1]:
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer,BertForSequenceClassification
import pytorch_lightning as pl

#日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,num_labels=2
)
bert_sc = bert_sc.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
text_list = [
    "この映画は面白かった。",
    "この映画の最後にはがっかりさせられた。",
    "この映画を見て幸せな気持ちになった。"
]
label_list = [1,0,1]

#データの符号化
encoding = tokenizer(
    text_list,
    padding='longest',
    return_tensors='pt'
)
encoding = { k: v.cuda() for k, v in encoding.items()}
labels = torch.tensor(label_list).cuda()

#推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits#分類スコア
labels_predicted = scores.argmax(-1)#スコアが最も値ラベル
num_correct = (labels_predicted==labels).sum().item()#正解数
accuracy = num_correct/labels.size(0)#精度

print("# scoresのサイズ:")
print(scores.size())
print("# predicted labels:")
print(labels_predicted)
print("# accuracy:")
print(accuracy)

# scoresのサイズ:
torch.Size([3, 2])
# predicted labels:
tensor([1, 0, 1], device='cuda:0')
# accuracy:
1.0


In [4]:
#符号化
encoding = tokenizer(
    text_list,
    padding='longest',
    return_tensors='pt'
)
encoding['labels']=torch.tensor(label_list)#入力にラベルを加える
encoding = {k: v.cuda() for k, v in encoding.items()}

#ロスの計算
output = bert_sc(**encoding)
loss = output.loss#損失の取得
print(loss)

tensor(0.6263, device='cuda:0', grad_fn=<NllLossBackward0>)


In [5]:
#データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

#ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz

'wget' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。
tar: Error opening archive: Failed to open 'ldcc-20140209.tar.gz'


In [6]:
#データローダの作成
dataset_for_loader = [
    #data:2次元テンソルのデータ　labels：データのクラス
    {'data':torch.tensor([0,1]),'labels':torch.tensor(0)},
    {'data':torch.tensor([2,3]),'labels':torch.tensor(1)},
    {'data':torch.tensor([4,5]),'labels':torch.tensor(2)},
    {'data':torch.tensor([6,7]),'labels':torch.tensor(3)},
]
#リストで4つあるデータを2つのデータ(2バッチ)で分割
loader = DataLoader(dataset_for_loader,batch_size=2)

#データセットからミニバッチを取り出す
for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)
    ##ファインチューニングではここでミニバッチごとの処理を行う

# batch 0
{'data': tensor([[0, 1],
        [2, 3]]), 'labels': tensor([0, 1])}
# batch 1
{'data': tensor([[4, 5],
        [6, 7]]), 'labels': tensor([2, 3])}


In [7]:
#データをシャッフルしてミニバッチ単位に分割
loader = DataLoader(dataset_for_loader,batch_size=2,shuffle=True)

for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)

# batch 0
{'data': tensor([[6, 7],
        [4, 5]]), 'labels': tensor([3, 2])}
# batch 1
{'data': tensor([[2, 3],
        [0, 1]]), 'labels': tensor([1, 0])}


In [8]:
#全記事の文章データを取得して前処理

#カテゴリーのリスト
category_list = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]

#トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

#各データの形式を整える
max_length = 128
dataset_for_loader = []
for label,category in enumerate(tqdm(category_list)):
    for file in glob.glob(f'./text/{category}/{category}*'):
        lines = open(file,encoding='utf-8').read().splitlines()
        text = '\n' .join(lines[3:])#ファイルの4行目から抜き出す
        encoding = tokenizer(
            text,
            max_length=max_length,
            padding='max_length',
            truncation=True
        )
        encoding['labels'] = label #ラベルを追加
        encoding = {k:torch.tensor(v) for k, v in encoding.items()}
        dataset_for_loader.append(encoding)

100%|██████████| 9/9 [00:44<00:00,  4.95s/it]


In [9]:
print(dataset_for_loader[0])

{'input_ids': tensor([    2,  2340, 19693, 10585, 28459,    35,  6692, 28493,    13,   501,
           62,   101,    37,     8,   569,   335,     5,    51,     7,     9,
         1040,     5,   616,     9,  2941,    18,  5602,   501,    20,    16,
         4027, 10531,   140,    36,    73, 30020, 28457, 25127,    38,  1080,
            5,    53,    28,   707,     5,    12,     9,    80,  3635,   205,
           29,  2935,   604,  5846,  6503,    11,  4722,    16,   861,    13,
            6, 12272, 24050,  2079,    11,    26,    62,    45,    28,  2451,
           80,     8,    36, 24050,    14,    31,  1058,    75, 11218, 10531,
         3676,   542,     5, 22130,     6,  5408,    16,  4831,    80,    29,
           18,  7045,    26, 28456,  4799,   900,     6,   569,   335,     9,
         1704,  1277,    15,  3318,  2575,    29,  2935,  5233,    75,    13,
         3472,   459,    12,  8585,  3171,   312,  3676,   542, 22130,   241,
            5,   709, 28696,  2180,    14, 12959, 

In [10]:
#データセットを分割
random.shuffle(dataset_for_loader)#ランダムにシャッフル
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train]#学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val]#検証データ
dataset_test = dataset_for_loader[n_train+n_val:]#テストデータ

#データセットからデータローダを作成
dataloader_train = DataLoader(
    dataset_train,batch_size=32,shuffle=True
)
datasloader_val = DataLoader(dataset_val,batch_size=256)
dataloader_test = DataLoader(dataset_test,batch_size=256)

In [26]:
#文章分類モデルの定義
from transformers import BertForSequenceClassification, BertConfig
class BertSewuenceClassification_pl(pl.LightningModule):
    
    def __init__(self,model_name,num_labels,lr):
        '''
        model_name:使用する事前学習済みモデル
        num_labels:分類のクラス数
        lr：学習率
        '''
        super().__init__()
        #後からモデルのハイパーパラメータを確認
        self.save_hyperparameters()
        
        #BertForSequenceClassificationモデルの初期化
        '''
        model_name：使用するBERTの事前学習済みモデルの名前
        num_class：分類タスクのクラス数
        '''
        config = BertConfig.from_pretrained(model_name, num_labels=num_labels)
        self.bert_sc = BertForSequenceClassification.from_pretrained(model_name, config=config)
        
    def training_step(self,batch,batch_idx):
        '''
        batch：データローダーから取得される1バッチ分のデータ
        batch_idx：現在のバッチ番号
        '''
        #バッチ内のモデルの出力
        output = self.bert_sc(**batch)
        #バッチ内の損失値
        loss = output.loss
        self.log('train_loss',loss)
        return loss
        
    def validation_step(self,batch,batch_idx):
        '''
        batch：データローダーから取得される1バッチ分のデータ
        batch_idx：現在のバッチ番号
        '''
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss',val_loss)
        
    def test_step(self,batch,batch_idx):
        '''
        batch：データローダーから取得される1バッチ分のデータ
        batch_idx：現在のバッチ番号
        '''
        #テストデータのバッチ全体を表す辞書から正解ラベルを取り出す
        labels = batch.pop('labels')
        #事前に定義された分類タスク用のBERTモデルにbatchを渡し、出力を取得
        output = self.bert_sc(**batch)
        #最もスコアが高いクラスのインデックスを取得
        labels_predicted = output.logits.argmax(-1)
        #予測されたクラスと正解ラベルを比較し、一致するかどうかをブール値で取得
        num_correct = (labels_predicted == labels).sum().item()
        #精度を計算
        accuracy = num_correct/labels.size(0)
        self.log('accuracy',accuracy)
        
    #Adamで勾配を計算する
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=self.hparams.lr)
        

In [27]:
#トレーニング中に特定の条件（モニタリングするメトリクス）を基にモデルを保存
checkpoint = pl.callbacks.ModelCheckpoint(
    #モニタリングするメトリクスとして、バリデーション損失を指定
    monitor='val_loss',
    #モニタリングするメトリクスの最小値を基準にモデルを保存
    mode='min',
    #最良のモデル（val_lossが最小のモデル）1つのみを保存
    save_top_k=1,
    #モデルの重み（パラメータ）のみを保存
    save_weights_only=True,
    #保存先ディレクトリを指定
    dirpath='model/',
)

#トレーニング、バリデーション、テストのループを自動的に処理
trainer = pl.Trainer(
    #使用するGPUの数を指定
    accelerator="gpu",  # GPUを使用
    devices=1,  
    #トレーニングを実行するエポック（データセット全体の反復回数）の最大数
    max_epochs=10,
    #トレーニング中に実行するコールバック（追加処理）のリストを指定
    callbacks=[checkpoint]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [28]:
#モデルの生成
'''
MODEL_NAME：使用する事前学習済みのBERTモデルの名前
num_labels：分類クラスの数
lr：学習率
'''
model = BertSewuenceClassification_pl(
    MODEL_NAME,num_labels=9,lr=1e-5
)

#トレーニング開始
trainer.fit(model,dataloader_train,datasloader_val)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                          | Params | Mode
-----------------------------------------------------------------
0 | bert_sc | BertForSequenceClassification | 110 M  | eval
-----------------------------------------------------------------
110 M     Train

Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  7.09it/s]

C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 139/139 [00:38<00:00,  3.61it/s, v_num=0]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 139/139 [00:38<00:00,  3.62it/s, v_num=0]   
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 139/139 [00:38<00:00,  3.61it/s, v_num=0]   
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 139/139 [00:38<00:00,  3.61it/s, v_num=0]   
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 139/139 [00:38<00:00,  3.59it/s, v_num=0]   
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 139/139 [00:38<00:00,  3.58it/s, v_num=0]   
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|██████████| 139/139 [00:38<00:00,  3.60it/s, v_num=0]   
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 7: 100%|██████████| 139/139 [00:38<00:00,  3.59it/s, v_num=0]   
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 8: 100%|██████████| 139/139 [00:38<00:00,  3.59it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 139/139 [00:42<00:00,  3.27it/s, v_num=0]


In [31]:
best_model_path = checkpoint.best_model_path#ベストモデルのファイル
print('ベストモデルのファイル:',checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失:',checkpoint.best_model_score)

ベストモデルのファイル: C:\Users\kinar\Desktop\natural_language_processing\model\epoch=4-step=695.ckpt
ベストモデルの検証データに対する損失: tensor(0.3802, device='cuda:0')


In [32]:
#学習時の学習データや検証データに対する損失の値の変化をグラフで表示
%load_ext tensorboard
%tensorboard --logdir ./

In [35]:
#テストデータで評価
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy：{test[0]["accuracy"]:.2f}')

C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\pytorch_lightning\trainer\connectors\checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at C:\Users\kinar\Desktop\natural_language_processing\model\epoch=4-step=695.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at C:\Users\kinar\Desktop\natural_language_processing\model\epoch=4-step=695.ckpt
C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the 

Testing DataLoader 0: 100%|██████████| 6/6 [00:03<00:00,  1.57it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.8690637946128845     │
└───────────────────────────┴───────────────────────────┘

Accuracy：0.87


In [36]:
#PyTorch Lightningモデルのロード
model = BertSewuenceClassification_pl.load_from_checkpoint(
    best_model_path
)

#Transformers対応モデルを./model_transformersに保存
model.bert_sc.save_pretrained('./model_trainformers')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
#ファインチューニングしたモデルを読み込む
bert_sc = BertForSequenceClassification.from_pretrained(
    './model_trainformers/'
)